## This is the main script where all simulations are launched
#### To add a new strategy to the competition, add it to the end of strategies.py . You can use all variables which are imported from lib.

In [ ]:
import lib
import strategies

competition = {fname: f for fname, f in strategies.__dict__.items() if callable(f)}

# Select only a part to run in the competition
competition = {k:v for k,v in competition.items() if k in ["buy", "buy_random"]}

for k, v in lib.run_competition(competition).items():
    print(k)
    print(v["stats"])


In [ ]:
#from scipy.optimize import minimize 
#minimize(f, x0, args=(a, b, c))

from scipy import optimize
import functools

#buy_if_success_ratio_geq_x_set = functools.partial(buy_if_success_ratio_geq_x, x=0.02)
def buy_if_success_ratio_geq_x_set(x): return buy_if_success_ratio_geq_x(x, 0.02)


#eval_f_over_prior(buy_if_success_ratio_geq_x_set, method="exact")["stats"]
list(run_competition({"a": buy_if_success_ratio_geq_x_set})["a"].keys())#["stats"]
#minimum = optimize.fmin(f, 1)
